# training data clean

In [1]:
import matplotlib.pyplot as plt
import us as usStates
import pandas as pd
import numpy as np
from numpy import savetxt
from datetime import datetime, timedelta
import sys
import os
import matplotlib.pyplot as plt
from sklearn import datasets
import json

%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=sys.maxsize)


# def days(d1, d2):
#     d1 = datetime.strptime(d1, "%m/%d/%Y")
#     d2 = datetime.strptime(d2, "%m/%d/%Y")
#     return abs((d2 - d1).days)+1

Get the Gleam predictions (point esitimate) and calculate the ML Gleam predictions (23 quantiles) 
for the next four week

In [2]:
f = open('../configs/data_cfg/data_cfg.json')
file = json.load(f)
gleam_path = file['gleam_path']
result_path = file['result_path']

In [3]:
df_origin = pd.read_csv(gleam_path+'2021-01-18-MOBS-GLEAM_COVID.csv')
df_origin = df_origin.fillna('NA')
df = df_origin.sort_values(by=['location','target','quantile'])
df1 = df.drop('forecast_date', axis=1)
print(df1.shape)
df2 = df1[(df1.target != '1 wk ahead cum death') & (df1.target != '2 wk ahead cum death')
   & (df1.target != '3 wk ahead cum death')& (df1.target != '4 wk ahead cum death')]
df2 = df1.loc[df1['target'].isin(['1 wk ahead inc death','2 wk ahead inc death',
                                 '3 wk ahead inc death','4 wk ahead inc death'])]
df3 = df2.loc[df2['location'] != 'US']
df4 = df3.loc[df3['type'] == 'point']
print(df2.shape)
print(df3.shape)
print(df4.shape)

(9792, 6)
(4896, 6)
(4800, 6)
(200, 6)


In [4]:
df_1week = df4.loc[df4['target_end_date'] == '2021-01-23']
df_2week = df4.loc[df4['target_end_date'] == '2021-01-30']
df_3week = df4.loc[df4['target_end_date'] == '2021-02-06']
df_4week = df4.loc[df4['target_end_date'] == '2021-02-13']

df_1week = df_1week.drop(['target','target_end_date','type','quantile'], axis=1)
df_2week = df_2week.drop(['target','target_end_date','type','quantile'], axis=1)
df_3week = df_3week.drop(['target','target_end_date','type','quantile'], axis=1)
df_4week = df_4week.drop(['target','target_end_date','type','quantile'], axis=1)


Gleam_week1 = np.expand_dims(df_1week['value'].to_numpy(),axis=1)
Gleam_week2 = np.expand_dims(df_2week['value'].to_numpy(),axis=1)
Gleam_week3 = np.expand_dims(df_3week['value'].to_numpy(),axis=1)
Gleam_week4 = np.expand_dims(df_4week['value'].to_numpy(),axis=1)

print(Gleam_week1.shape)
print(Gleam_week2.shape)
print(Gleam_week3.shape)
print(Gleam_week4.shape)

(50, 1)
(50, 1)
(50, 1)
(50, 1)


In [5]:
ML_residual = np.load(result_path+'ensembledata.npy')
# ML_residual = np.load('ensembledata_torchts.npy')
    
ML_residual1 = ML_residual[0]
ML_residual2 = ML_residual[1]
ML_residual3 = ML_residual[2]
ML_residual4 = ML_residual[3]

print(ML_residual1.shape)
print(ML_residual2.shape)
print(ML_residual3.shape)
print(ML_residual4.shape)

(50, 23)
(50, 23)
(50, 23)
(50, 23)


In [6]:
ML_Gleam_death1 = np.flip(Gleam_week1 - ML_residual1,axis =1)
ML_Gleam_death2 = np.flip(Gleam_week2 - ML_residual2,axis =1)
ML_Gleam_death3 = np.flip(Gleam_week3 - ML_residual3,axis =1)
ML_Gleam_death4 = np.flip(Gleam_week4 - ML_residual4,axis =1)

print(22*50-sum(sum(np.diff(ML_Gleam_death1)>=0)))
print(22*50-sum(sum(np.diff(ML_Gleam_death2)>=0)))
print(22*50-sum(sum(np.diff(ML_Gleam_death3)>=0)))
print(22*50-sum(sum(np.diff(ML_Gleam_death4)>=0)))

print(np.sum(np.array(ML_Gleam_death1) < 0, axis=0))
print(np.sum(np.array(ML_Gleam_death2) < 0, axis=0))
print(np.sum(np.array(ML_Gleam_death3) < 0, axis=0))
print(np.sum(np.array(ML_Gleam_death4) < 0, axis=0))

#set to positive
ML_Gleam_death1 = np.maximum(ML_Gleam_death1,0)
ML_Gleam_death2 = np.maximum(ML_Gleam_death2,0)
ML_Gleam_death3 = np.maximum(ML_Gleam_death3,0)
ML_Gleam_death4 = np.maximum(ML_Gleam_death4,0)

print(np.sum(np.array(ML_Gleam_death1) < 0, axis=0))
print(np.sum(np.array(ML_Gleam_death2) < 0, axis=0))
print(np.sum(np.array(ML_Gleam_death3) < 0, axis=0))
print(np.sum(np.array(ML_Gleam_death4) < 0, axis=0))


0
0
0
0
[15 15 13 11  9  8  6  4  3  3  0  0  0  0  0  0  0  0  0  0  0  0  0]
[19 15 15 12 10  9  7  6  4  3  0  0  0  0  0  0  0  0  0  0  0  0  0]
[21 18 15 13 11  9  8  6  4  4  0  0  0  0  0  0  0  0  0  0  0  0  0]
[22 19 18 13 12 11  9  6  5  4  0  0  0  0  0  0  0  0  0  0  0  0  0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [7]:
#inc data use sorted ML Leam death
# add US. data
US_death1 = np.append(ML_Gleam_death1,np.expand_dims(np.sum(ML_Gleam_death1,axis = 0),axis=0),axis = 0)
US_death2 = np.append(ML_Gleam_death2,np.expand_dims(np.sum(ML_Gleam_death2,axis = 0),axis=0),axis = 0)
US_death3 = np.append(ML_Gleam_death3,np.expand_dims(np.sum(ML_Gleam_death3,axis = 0),axis=0),axis = 0)
US_death4 = np.append(ML_Gleam_death4,np.expand_dims(np.sum(ML_Gleam_death4,axis = 0),axis=0),axis = 0)

# copy 50% quantile for point
inc_death1 = np.insert(US_death1,23,US_death1[:,11],axis = 1)
inc_death2 = np.insert(US_death2,23,US_death2[:,11],axis = 1)
inc_death3 = np.insert(US_death3,23,US_death3[:,11],axis = 1)
inc_death4 = np.insert(US_death4,23,US_death4[:,11],axis = 1)

generate the submission file

In [8]:
# previous death
cumweek1 = df1.loc[(df1['target'] =='1 wk ahead cum death') & (df1['type'] == 'point')]
incweek1 = df1.loc[(df1['target'] =='1 wk ahead inc death')& (df1['type'] == 'point')]
cumweek_1 = cumweek1['value'].to_numpy()
incweek_1 = incweek1['value'].to_numpy()
previous_death = np.around(cumweek_1 - incweek_1).astype(int)
print(previous_death)

[  6119    229  11248   4293  33342   5363   6594   1016  24004  12291
    319   1605  20020   9287   4321   3489   3093   8080    507   6541
  13583  14669   5955   5481   6467   1088   1837   3761    927  20414
   2910  40806   8016   1403  10135   2952   1799  19143   2005   6108
   1633   8355  32428   1485    163   5706   3903   1761   5905    522
 395785]


In [9]:
# cum death
cum_death1 = np.add(inc_death1,np.expand_dims(previous_death,axis = 1))
cum_death2 = np.add(inc_death2,cum_death1)
cum_death3 = np.add(inc_death3,cum_death2)
cum_death4 = np.add(inc_death4,cum_death3)


In [10]:
# all the data
value_data = np.stack([cum_death1,inc_death1,cum_death2,inc_death2,
                             cum_death3,inc_death3,cum_death4,inc_death4],axis=1).flatten()
# add to the value column
print(value_data.shape)

(9792,)


In [11]:
df.drop(columns=['value'])
df['value'] = value_data

df = df.astype({'forecast_date': 'string','target': 'string', 'target_end_date': 'string', 'location': 'string',
          'type': 'string', 'location': 'string', 'value': 'float64'})



In [12]:
df.dtypes

forecast_date       string
target              string
target_end_date     string
location            string
type                string
quantile            object
value              float64
dtype: object

In [13]:
df.to_csv(gleam_path+'2021-01-18-UCSD_NEU-DeepGLEAM.csv',index=False)